In [1]:
#%reload_ext autoreload
#%autoreload 2

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)


In [2]:
import pickle
import seaborn as sns

import matplotlib.pyplot as plt
from dbspace.readout import ClinVect, decoder
import dbspace.readout.BR_DataFrame as BR_DataFrame

plt.rcParams["image.cmap"] = "tab10"

sns.set_context("paper")
sns.set(font_scale=4)
sns.set_style("white")

## MAJOR PARAMETERS for our partial biometric analysis
do_pts = [
    "901",
    "903",
    "905",
    "906",
    "907",
    "908",
]  # Which patients do we want to include in this entire analysis?
test_scale = "pHDRS17"  # Which scale are we using as the measurement of the depression state? pHDRS17 = nHDRS (from paper) and is a patient-specific normalized HDRS


Importing from DBSpace.readout...


/dbspace/src/dbspace/readout/ClinVect.py:21: UserWarning: Warning...........Message
  warnings.warn("Warning...........Message")


In [3]:
# Initialize our Clinical Frame and load in our BR Frame

ClinFrame = ClinVect.CStruct("/data/clinical/mayberg_2013/clinical_vectors_all.json")
if test_scale == "mHDRS":
    ClinFrame.gen_mHDRS()
elif test_scale == "DSC":
    ClinFrame.gen_DSC()

frame_to_analyse = 'Chronic_Frame_Dec2022'
BRFrame = pickle.load(open(f"/tmp/{frame_to_analyse}.pickle","rb"))


/dbspace/src/dbspace/readout/ClinVect.py:231: UserWarning: No Stim Changes Metadata file (mat) provided.
  warnings.warn("No Stim Changes Metadata file (mat) provided.")


# Train, test, validate the weekly decoder

In [4]:
main_readout = decoder.weekly_decoderCV(
    BRFrame=BRFrame,
    ClinFrame=ClinFrame,
    pts=do_pts,
    clin_measure=test_scale,
    algo="ENR",
    alpha=-4,
    shuffle_null=False,
    FeatureSet="main",
    variance=False,
)  # main analysis is -3.4
main_readout.global_plotting = True
main_readout.filter_recs(rec_class="main_study")
main_readout.split_train_set(0.6)


Initialized the Weekly CV decoder
Running ENR_CV w/:0.8


In [7]:
main_readout.train_setup()
optimal_alpha = (
    main_readout._path_slope_regression()
)  # suppress_vars=1/40)#,override_alpha=2**-3) #suppress_vars = 1 works well if we're not doing THarm analysis
main_readout.train_model()


Performing Training Setup for Weekly Decoder


AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
main_readout.plot_decode_CV()


In [ ]:
main_readout.test_setup()
main_readout.test_model()

main_readout.plot_test_timecourse()

In [ ]:
main_readout.plot_test_stats()

In [ ]:
main_readout.plot_test_regression_figure()
# main_readout.plot_combo_paths()
